In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#leer el data
data = pd.read_csv('octavos.csv')
data.head()

,Equipo 1,Agr.,Equipo 2,Ida,Vuelta,Temporada
0,Leipzig,1–8,Manchester City,1–1,0–7,22/23
1,Brujas,1–7,Benfica,0–2,1–5,22/23
2,Liverpool,2–6,Real Madrid,2–5,0–1,22/23
3,Milan,1–0,Tottenham Hotspur,1–0,0–0,22/23
4,Eintracht Fráncfort,0–5,Napoli,0–2,0–3,22/23
